# Necessary Imports and Settings

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import transformers
import torch
import os
import nltk
import pandas as pd
import torch
import numpy as np
from jinja2 import Template
import xmltodict
import pickle
from collections import defaultdict
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import time
import copy

import sys
sys.path.append('/scratch/users/bozyurt20/hpc_run/utilities')
sys.path.append("/scratch/users/bozyurt20/hpc_run/blobs/")
from util_research import *

#from toy_dataset import contexts

max_len = 512
num_layers = 24
d_model = 4096

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0pp", truncation_side="right", add_prefix_space=True)

In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0pp", device_map="balanced", load_in_8bit=True)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /kuacc/users/bozyurt20/.conda/envs/hf/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda110.so...


In [12]:
names_1 = ["John", "Harry", "Andrew"]
names_2 = ["Henry", "David", "Sophia"]
names_3 = ["Olivia", "Emma", "Lisa"]

cities_1 = ["London", "Paris", "Oslo"]
cities_2 = ["Sydney", "Cairo", "Seoul"]
cities_3 = ["Istanbul", "Beijing", "Rome"]

class DataSample_ThreeSentences():
    def __init__(self, name_1, name_2, name_3, city_1, city_2, city_3):
        self.name_1 = name_1
        self.name_2 = name_2
        self.name_3 = name_3
        self.city_1 = city_1
        self.city_2 = city_2
        self.city_3 = city_3
        self.context_1 = name_1 + " travelled to " + city_1 + ". " + name_2 + " travelled to " + city_2 + ". " + name_3 + " travelled to " + city_3 + ". Where did " + name_3 + " travel to?"
        self.context_2 = name_1 + " travelled to " + city_1 + ". " + name_2 + " travelled to " + city_2 + ". " + name_3 + " travelled to. Where did " + name_3 + " travel to?"
        self.context_3 = name_1 + " travelled to " + city_1 + ". " + name_2 + " travelled to " + city_2 + ". " + name_3 + " travelled to ???. Where did " + name_3 + " travel to?"
  
    def add_context_encoding_1(self, encoding):
        self.context_encoding_1 = encoding
    def add_context_encoding_2(self, encoding):
        self.context_encoding_2 = encoding
    def add_context_encoding_3(self, encoding):
        self.context_encoding_3 = encoding
    def add_generation_1(self, model_out):
        self.model_out_1 = model_out
    def add_generation_2(self, model_out):
        self.model_out_2 = model_out
    def add_generation_3(self, model_out):
        self.model_out_3 = model_out
        
class ThreeSentenceRewriteResult():
    def __init__ (self, context_1, context_2, context_3, generation_rewritten_2, generation_rewritten_3, answer):
        
        self.context_1 = context_1
        self.context_2 = context_2
        self.context_3 = context_3
        #self.generation_1 = generation_1
        #self.generation_2 = generation_2
        #self.generation_3 = generation_3
        self.generation_rewritten_2 = generation_rewritten_2
        self.generation_rewritten_3 = generation_rewritten_3
        self.answer = answer

In [13]:
def case1_rewrite(data_samples, ids_to_move):
    
    case1_results = []

    for sample in data_samples:

        encoding_sample_1 = sample.context_encoding_1
        context_1 = sample.context_1
        context_2 = sample.context_2
        context_3 = sample.context_3        
        answer = sample.city_3

        special_hidden = encoding_sample_1.special_hidden_states # 24 x (1, T, d)
        len_input_ids = special_hidden[0].shape[1]
        special_reformatted = torch.zeros(num_layers, len_input_ids, d_model) # (24, T, d)
        for i, hidden in enumerate(special_hidden):
            special_reformatted[i:i+1, :, :] = hidden

        entities_hidden_states = special_reformatted[:, ids_to_move[0], :].unsqueeze(0) # 1, 24, d
        
        for tok_id in ids_to_move[1:]:
            entity_hidden_states = special_reformatted[:, tok_id, :].unsqueeze(0) # 1, 24, d
            entities_hidden_states = torch.cat((entities_hidden_states,
                                                entity_hidden_states), dim=0)

        input_ids = tokenizer.encode(context_2, return_tensors="pt").to(model.encoder.device)
        generation_rewritten_2 = model.generate(input_ids=input_ids, 
                                             max_new_tokens=10,
                                             entity_hidden_states=entities_hidden_states,
                                             entity_inds=ids_to_move,
                                             return_dict_in_generate=True, 
                                             output_scores=True)

        input_ids = tokenizer.encode(context_3, return_tensors="pt").to(model.encoder.device)
        generation_rewritten_3 = model.generate(input_ids=input_ids, 
                                             max_new_tokens=10,
                                             entity_hidden_states=entities_hidden_states,
                                             entity_inds=ids_to_move,
                                             return_dict_in_generate=True, 
                                             output_scores=True)
        case1_results.append(ThreeSentenceRewriteResult(context_1, context_2, context_3, generation_rewritten_2, generation_rewritten_3, answer))
        
    return case1_results

In [14]:
def case2_rewrite(data_samples, ids_to_move):

    case2_results = []

    for sample in data_samples:
        
        encoding_sample_1 = sample.context_encoding_1
        encoding_sample_2 = sample.context_encoding_2
        encoding_sample_3 = sample.context_encoding_3
        context_1 = sample.context_1
        context_2 = sample.context_2
        context_3 = sample.context_3        
        answer = sample.city_3

        encoding_sample_2_copy = copy.deepcopy(encoding_sample_2)

        for i in ids_to_move:
            encoding_sample_2_copy.last_hidden_state[:,i:i+1,:] = encoding_sample_1.last_hidden_state[:,i:i+1,:]
        
        generation_rewritten_2 = model.generate(encoder_outputs=encoding_sample_2_copy, 
                                               max_new_tokens=10,
                                               return_dict_in_generate=True, 
                                               output_scores=True)
        
        encoding_sample_3_copy = copy.deepcopy(encoding_sample_3)

        for i in ids_to_move:
            encoding_sample_3_copy.last_hidden_state[:,i:i+1,:] = encoding_sample_1.last_hidden_state[:,i:i+1,:]
        
        generation_rewritten_3 = model.generate(encoder_outputs=encoding_sample_3_copy, 
                                               max_new_tokens=10,
                                               return_dict_in_generate=True, 
                                               output_scores=True)
        
        case2_results.append(ThreeSentenceRewriteResult(context_1, context_2, context_3, generation_rewritten_2, generation_rewritten_3, answer))
        
    return case2_results

# Data Pre-Encodings

In [7]:
data_points_three_sentences_case1 = []
for name_1 in names_1:
    for name_2 in names_2:
        for name_3 in names_3:
            for city_1 in cities_1:
                for city_2 in cities_2:
                    for city_3 in cities_3:
                        sample = DataSample_ThreeSentences(name_1, name_2, name_3, city_1, city_2, city_3)
                        
                        context_1 = sample.context_1
                        context_2 = sample.context_2
                        context_3 = sample.context_3
                        
                        context_1_tokens = tokenizer.encode(context_1, return_tensors="pt").to(model.encoder.device)
                        context_2_tokens = tokenizer.encode(context_2, return_tensors="pt").to(model.encoder.device)
                        context_3_tokens = tokenizer.encode(context_3, return_tensors="pt").to(model.encoder.device)
                        with torch.no_grad():
                            encoded_context_1 = model.encoder(context_1_tokens, output_special=True, output_hidden_states=True)
                            encoded_context_2 = model.encoder(context_2_tokens, output_special=True, output_hidden_states=True)
                            encoded_context_3 = model.encoder(context_3_tokens, output_special=True, output_hidden_states=True)
                            
                        """with torch.no_grad():
                            model_out_1 = model.generate(input_ids=context_1_tokens, 
                                         max_new_tokens=10,
                                         return_dict_in_generate=True, 
                                         output_scores=True)
                            model_out_2 = model.generate(input_ids=context_2_tokens, 
                                         max_new_tokens=10,
                                         return_dict_in_generate=True, 
                                         output_scores=True)
                            model_out_3 = model.generate(input_ids=context_3_tokens, 
                                         max_new_tokens=10,
                                         return_dict_in_generate=True, 
                                         output_scores=True)"""

                        sample.add_context_encoding_1(encoded_context_1)
                        sample.add_context_encoding_2(encoded_context_2)
                        sample.add_context_encoding_3(encoded_context_3)
                        
                        """sample.add_generation_1(model_out_1)
                        sample.add_generation_2(model_out_2)
                        sample.add_generation_3(model_out_3)"""
                        
                        data_points_three_sentences_case1.append(sample)

In [8]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/NewThreeSentenceDataEncodings/data_points_three_sentences.txt", "wb") as f:
    pickle.dump(data_points_three_sentences_case1, f)

In [4]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/NewThreeSentenceDataEncodings/data_points_three_sentences.txt", "rb") as f:
    data_points_three_sentences_case1 = pickle.load(f)

# Case 1 Rewrite

In [32]:
tokenizer.tokenize(context_2)

['▁Andrew',
 '▁',
 'travelled',
 '▁to',
 '▁Oslo',
 '.',
 '▁Sophia',
 '▁',
 'travelled',
 '▁to',
 '▁Seoul',
 '.',
 '▁Lisa',
 '▁',
 'travelled',
 '▁to',
 '.',
 '▁Where',
 '▁did',
 '▁Lisa',
 '▁travel',
 '▁to',
 '?']

In [15]:
ids_to_move = [9] # middle _to token

case1_middle_to_results = case1_rewrite(data_points_three_sentences_case1, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case1_middle_to_results.txt", "wb") as f:
    pickle.dump(case1_middle_to_results, f)

In [6]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case1_middle_to_results.txt", "rb") as f:
    case1_middle_to_results = pickle.load(f)

In [16]:
ids_to_move = [16]

case1_loc_results = case1_rewrite(data_points_three_sentences_case1, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case1_loc_results.txt", "wb") as f:
    pickle.dump(case1_loc_results, f)

In [17]:
ids_to_move = [12, 13, 14, 15]

case1_allexceptloc_results = case1_rewrite(data_points_three_sentences_case1, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case1_allexceptloc_results.txt", "wb") as f:
    pickle.dump(case1_allexceptloc_results, f)

In [18]:
ids_to_move = [12, 13, 14, 15, 16]

case1_alltok_results = case1_rewrite(data_points_three_sentences_case1, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case1_alltok_results.txt", "wb") as f:
    pickle.dump(case1_alltok_results, f)

KeyboardInterrupt: 

In [ ]:
ids_to_move = list(range(16))

case1_all3sentexceptlastloc_results = case1_rewrite(data_points_three_sentences_case1, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case1_all3sentexceptlastloc_results.txt", "wb") as f:
    pickle.dump(case1_all3sentexceptlastloc_results, f)

# Case 2 Rewrite

In [ ]:
ids_to_move = [9] # middle _to token

case2_middle_to_results = case2_rewrite(data_points_three_sentences_case1, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case2_middle_to_results.txt", "wb") as f:
    pickle.dump(case2_middle_to_results, f)

In [7]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case2_middle_to_results.txt", "rb") as f:
    case2_middle_to_results = pickle.load(f)

In [ ]:
ids_to_move = [16]

case2_loc_results = case2_rewrite(data_points_three_sentences_case1, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case2_loc_results.txt", "wb") as f:
    pickle.dump(case2_loc_results, f)

In [ ]:
ids_to_move = [12, 13, 14, 15]

case2_allexceptloc_results = case2_rewrite(data_points_three_sentences_case1, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case2_allexceptloc_results.txt", "wb") as f:
    pickle.dump(case2_allexceptloc_results, f)

In [ ]:
ids_to_move = [12, 13, 14, 15, 16]

case2_alltok_results = case2_rewrite(data_points_three_sentences_case1, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case2_alltok_results.txt", "wb") as f:
    pickle.dump(case2_alltok_results, f)

In [ ]:
ids_to_move = list(range(16))

case2_all3sentexceptlastloc_results = case2_rewrite(data_points_three_sentences_case1, ids_to_move)

with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case2_all3sentexceptlastloc_results.txt", "wb") as f:
    pickle.dump(case2_all3sentexceptlastloc_results, f)

# Result Inspection

In [4]:
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case1_middle_to_results.txt", "rb") as f:
    case1_middle_to_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case1_loc_results.txt", "rb") as f:
    case1_loc_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case1_allexceptloc_results.txt", "rb") as f:
    case1_allexceptloc_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case1_alltok_results.txt", "rb") as f:
    case1_alltok_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case1_all3sentexceptlastloc_results.txt", "rb") as f:
    case1_all3sentexceptlastloc_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case2_middle_to_results.txt", "rb") as f:
    case2_middle_to_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case2_loc_results.txt", "rb") as f:
    case2_loc_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case2_allexceptloc_results.txt", "rb") as f:
    case2_allexceptloc_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case2_alltok_results.txt", "rb") as f:
    case2_alltok_results = pickle.load(f)
with open("/kuacc/users/bozyurt20/hpc_run/Sherlock Holmes/Rewrite/New Results/case2_all3sentexceptlastloc_results.txt", "rb") as f:
    case2_all3sentexceptlastloc_results = pickle.load(f)

In [19]:
def get_token_probability(model_out, token):
    
    global next_token_scores
    token_id = tokenizer.encode(token)[0]
    next_token_scores = torch.nn.functional.softmax(model_out.scores[0].float(), dim=-1)  # (batch_size * num_beams, vocab_size)
    probability = next_token_scores[0][token_id].item()
    
    _, indices = torch.sort(next_token_scores, descending=True)
    max_prob_ind = torch.where(indices[0] == token_id)[0].item() + 1
    
    return probability, max_prob_ind



In [20]:
def check_rewrite_results(results):
    
    sentence_2_acc = []
    sentence_3_acc = []

    sentence_rewritten_answer_1 = []
    sentence_rewritten_answer_2 = []

    generation_lens = defaultdict(int)

    probability_results = np.zeros((2, 3**6))
    sorted_results = np.zeros((2, 3**6))

    for i, result in enumerate(results):
        context_1 = result.context_1    
        context_2 = result.context_2
        context_3 = result.context_3
        generation_rewritten_2 = result.generation_rewritten_2
        generation_rewritten_3 = result.generation_rewritten_3
        answer = result.answer

        out_sequence_rewritten_2 = tokenizer.decode(generation_rewritten_2.sequences[0], skip_special_tokens=True)
        out_sequence_rewritten_3 = tokenizer.decode(generation_rewritten_3.sequences[0], skip_special_tokens=True)
        
        gen_rw_2, ind_rw2 = get_token_probability(generation_rewritten_2, answer)
        gen_rw_3, ind_rw3 = get_token_probability(generation_rewritten_3, answer)
        
        generation_lens[len(generation_rewritten_2.sequences[0])] += 1
        generation_lens[len(generation_rewritten_3.sequences[0])] += 1

        if answer in out_sequence_rewritten_2:
            sentence_2_acc.append(1)
        else:
            sentence_2_acc.append(0)
        if answer in out_sequence_rewritten_3:
            sentence_3_acc.append(1)
        else:
            sentence_3_acc.append(0)
            
        probability_results[:, i] = [gen_rw_2 , gen_rw_3]

        sorted_results[:, i] = [ind_rw2, ind_rw3]

    acc_2 = sum(sentence_2_acc)/len(sentence_2_acc)
    acc_3 = sum(sentence_3_acc)/len(sentence_3_acc)

    print("Accuracy of rewritten context, sentence 2:", acc_2)
    print("Accuracy of rewritten context, sentence 3:", acc_3)
    print()
    print("Probability of generating the answer tokens: ", np.average(probability_results, axis=-1).tolist())
    print("Token generation index:", np.average(sorted_results, axis=-1).tolist())
    
    return acc_2, acc_3, generation_lens, probability_results, sorted_results

## Case 1

In [21]:
acc_2, acc_3, generation_lens, probability_results, sorted_results = check_rewrite_results(case1_middle_to_results)
print()
print(generation_lens)

Accuracy of rewritten context, sentence 2: 0.0
Accuracy of rewritten context, sentence 3: 0.0

Probability of generating the answer tokens:  [0.011085331013195589, 0.011360536589426472]
Token generation index: [20.77914951989026, 21.425240054869686]

defaultdict(<class 'int'>, {3: 1402, 4: 56})


In [33]:
for i, result in enumerate(case1_allexceptloc_results):
    context_1 = result.context_1    
    context_2 = result.context_2
    context_3 = result.context_3
    generation_rewritten_2 = result.generation_rewritten_2
    generation_rewritten_3 = result.generation_rewritten_3
    answer = result.answer

    out_sequence_rewritten_2 = tokenizer.decode(generation_rewritten_2.sequences[0], skip_special_tokens=True)
    out_sequence_rewritten_3 = tokenizer.decode(generation_rewritten_3.sequences[0], skip_special_tokens=True)
    
    print(context_1)
    print(context_2)
    print(out_sequence_rewritten_2)
    print(context_3)
    print(out_sequence_rewritten_3)
    print()

John travelled to London. Henry travelled to Sydney. Olivia travelled to Istanbul. Where did Olivia travel to?
John travelled to London. Henry travelled to Sydney. Olivia travelled to. Where did Olivia travel to?
Sydney
John travelled to London. Henry travelled to Sydney. Olivia travelled to ???. Where did Olivia travel to?
???

John travelled to London. Henry travelled to Sydney. Olivia travelled to Beijing. Where did Olivia travel to?
John travelled to London. Henry travelled to Sydney. Olivia travelled to. Where did Olivia travel to?
Sydney
John travelled to London. Henry travelled to Sydney. Olivia travelled to ???. Where did Olivia travel to?
Sydney

John travelled to London. Henry travelled to Sydney. Olivia travelled to Rome. Where did Olivia travel to?
John travelled to London. Henry travelled to Sydney. Olivia travelled to. Where did Olivia travel to?
Sydney
John travelled to London. Henry travelled to Sydney. Olivia travelled to ???. Where did Olivia travel to?
Sydney

John t

In [22]:
acc_2, acc_3, generation_lens, probability_results, sorted_results = check_rewrite_results(case1_loc_results)
print()
print(generation_lens)

Accuracy of rewritten context, sentence 2: 1.0
Accuracy of rewritten context, sentence 3: 1.0

Probability of generating the answer tokens:  [0.9655522813685801, 0.9739434541661717]
Token generation index: [1.0, 1.0]

defaultdict(<class 'int'>, {3: 1458})


In [23]:
acc_2, acc_3, generation_lens, probability_results, sorted_results = check_rewrite_results(case1_allexceptloc_results)
print()
print(generation_lens)

Accuracy of rewritten context, sentence 2: 0.0
Accuracy of rewritten context, sentence 3: 0.0

Probability of generating the answer tokens:  [0.009863439364631377, 0.006290741682733538]
Token generation index: [35.554183813443075, 28.84636488340192]

defaultdict(<class 'int'>, {3: 864, 4: 497, 8: 91, 7: 6})


In [26]:
acc_2, acc_3, generation_lens, probability_results, sorted_results = check_rewrite_results(case1_alltok_results)
print()
print(generation_lens)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 1.0
Accuracy of rewritten context, city 2: 0.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [0.9570660140615759, 3.609393726884016e-05]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [-0.0007771709818898896, -9.501372659354153e-08], [0.9570559897050467, -0.9807976715550853]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [1.0, 274.91495198902606]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [0.0, 4.917695473251029], [-191.77366255144034, 273.91495198902606]]


In [27]:
acc_2, acc_3, generation_lens, probability_results, sorted_results = check_rewrite_results(case1_all3sentexceptlastloc_results)
print()
print(generation_lens)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 0.0
Accuracy of rewritten context, city 2: 1.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [9.341803144641105e-06, 0.980047610725068]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [-0.9578338432403212, 0.9800114217740725], [-6.825533846251811e-07, -0.0007861547672863032]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [198.22085048010973, 1.0]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [197.22085048010973, -268.997256515775], [5.44718792866941, 0.0]]


## Case 2

In [28]:
acc_2, acc_3, generation_lens, probability_results, sorted_results = check_rewrite_results(case2_middle_to_results)
print()
print(generation_lens)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 0.0
Accuracy of rewritten context, city 2: 1.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [1.0093023461045406e-05, 0.9807449892388123]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [-0.9578330920200048, 0.9807088002878168], [6.866693177911862e-08, -8.877625354196473e-05]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [193.90946502057614, 1.0]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [192.90946502057614, -268.997256515775], [1.1358024691358024, 0.0]]


In [29]:
acc_2, acc_3, generation_lens, probability_results, sorted_results = check_rewrite_results(case2_loc_results)
print()
print(generation_lens)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 1.0
Accuracy of rewritten context, city 2: 0.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [0.9593255508420235, 3.8994317712524644e-05]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [0.0014823657985577367, 2.8053667170909503e-06], [0.9593155264854943, -0.9807947711746418]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [1.0, 230.88477366255145]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [0.0, -39.1124828532236], [-191.77366255144034, 229.88477366255145]]


In [30]:
acc_2, acc_3, generation_lens, probability_results, sorted_results = check_rewrite_results(case2_allexceptloc_results)
print()
print(generation_lens)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 0.0
Accuracy of rewritten context, city 2: 1.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [1.0353784404255143e-05, 0.9796839546110732]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [-0.9578328312590615, 0.9796477656600777], [3.294278749888563e-07, -0.0011498108812810952]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [186.86008230452674, 1.0]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [185.86008230452674, -268.997256515775], [-5.91358024691358, 0.0]]


In [31]:
acc_2, acc_3, generation_lens, probability_results, sorted_results = check_rewrite_results(case2_alltok_results)
print()
print(generation_lens)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 1.0
Accuracy of rewritten context, city 2: 0.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [0.9574766288241895, 3.673426079406445e-05]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [-0.0003665562192762503, 5.453097986307557e-07], [0.9574666044676603, -0.9807970312315601]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [1.0, 262.22770919067216]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [0.0, -7.769547325102881], [-191.77366255144034, 261.22770919067216]]


In [32]:
acc_2, acc_3, generation_lens, probability_results, sorted_results = check_rewrite_results(case2_all3sentexceptlastloc_results)
print()
print(generation_lens)

Accuracy of context 1, city1: 1.0
Accuracy of context 2, city2: 1.0
Accuracy of rewritten context, city 1: 0.0
Accuracy of rewritten context, city 2: 1.0

Probability of generating the answer tokens:  [[0.9578431850434658, 3.6188950995433694e-05], [1.0024356529266287e-05, 0.9808337654923542], [9.973773307080501e-06, 0.9791468326283417]]
Probability of generating the answer tokens, difference:  [[0.9578331606869366, -0.9807975765413588], [-0.9578332112701587, 0.9791106436773462], [-5.058322218578618e-08, -0.0016869328640125416]]
Token generation index: [[1.0, 269.997256515775], [192.77366255144034, 1.0], [196.14677640603566, 1.0]]
Token generation index, difference: [[-191.77366255144034, 268.997256515775], [195.14677640603566, -268.997256515775], [3.373113854595336, 0.0]]


In [15]:
import sys
sys.path.append('/kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/models/')
cp modeling_t5.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/models/t5/
cp utils.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/generation/
cp modeling_outputs.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/

SyntaxError: invalid syntax (676971914.py, line 3)